In [1]:
import pandas as pd
import numpy as np
import torch
import scripts
from functools import lru_cache
import torchmetrics
from torch import nn
import optuna

/home/karolyi/miniconda/envs/project_thesis/lib/python3.13/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data loading

First we load the data. The basic idea is to create dictionaries with features associated to the drugs and cell-lines. In principle, the splits and the data shouldn't be changed

In [2]:
@lru_cache(maxsize = None)
def get_data(n_fold = 0, fp_radius = 2):
    smile_dict = pd.read_csv("data/smiles.csv", index_col=0)
    fp = scripts.FingerprintFeaturizer(R = fp_radius)
    drug_dict = fp(smile_dict.iloc[:, 1], smile_dict.iloc[:, 0])
    driver_genes = pd.read_csv("data/driver_genes.csv").loc[:, "symbol"].dropna()
    rnaseq = pd.read_csv("data/rnaseq_normcount.csv", index_col=0)
    driver_columns = rnaseq.columns.isin(driver_genes)
    filtered_rna = rnaseq.loc[:, driver_columns]
    tensor_exp = torch.Tensor(filtered_rna.to_numpy())
    cell_dict = {cell: tensor_exp[i] for i, cell in enumerate(filtered_rna.index.to_numpy())}
    data = pd.read_csv("data/GDSC12.csv", index_col=0)
    # default, remove data where lines or drugs are missing:
    data = data.query("SANGER_MODEL_ID in @cell_dict.keys() & DRUG_ID in @drug_dict.keys()")
    unique_cell_lines = data.loc[:, "SANGER_MODEL_ID"].unique()
    np.random.seed(420) # for comparibility, don't change it!
    np.random.shuffle(unique_cell_lines)
    folds = np.array_split(unique_cell_lines, 10)
    test_lines = folds[0]
    train_idxs = list(range(10))
    train_idxs.remove(n_fold)
    np.random.seed(420)
    validation_idx = np.random.choice(train_idxs)
    train_idxs.remove(validation_idx)
    train_lines = np.concatenate([folds[idx] for idx in train_idxs])
    validation_lines = folds[validation_idx]
    test_lines = folds[n_fold]
    train_data = data.query("SANGER_MODEL_ID in @train_lines")
    validation_data = data.query("SANGER_MODEL_ID in @validation_lines")
    test_data = data.query("SANGER_MODEL_ID in @test_lines")
    return (scripts.OmicsDataset_drugwise(cell_dict, drug_dict, train_data),
    scripts.OmicsDataset_drugwise(cell_dict, drug_dict, validation_data),
    scripts.OmicsDataset_drugwise(cell_dict, drug_dict, test_data))

In [3]:
@lru_cache(maxsize = None)
def get_data_original(n_fold = 0, fp_radius = 2):
    smile_dict = pd.read_csv("data/smiles.csv", index_col=0)
    fp = scripts.FingerprintFeaturizer(R = fp_radius)
    drug_dict = fp(smile_dict.iloc[:, 1], smile_dict.iloc[:, 0])
    driver_genes = pd.read_csv("data/driver_genes.csv").loc[:, "symbol"].dropna()
    rnaseq = pd.read_csv("data/rnaseq_normcount.csv", index_col=0)
    driver_columns = rnaseq.columns.isin(driver_genes)
    filtered_rna = rnaseq.loc[:, driver_columns]
    tensor_exp = torch.Tensor(filtered_rna.to_numpy())
    cell_dict = {cell: tensor_exp[i] for i, cell in enumerate(filtered_rna.index.to_numpy())}
    data = pd.read_csv("data/GDSC12.csv", index_col=0)
    # default, remove data where lines or drugs are missing:
    data = data.query("SANGER_MODEL_ID in @cell_dict.keys() & DRUG_ID in @drug_dict.keys()")
    unique_cell_lines = data.loc[:, "SANGER_MODEL_ID"].unique()
    np.random.seed(420) # for comparibility, don't change it!
    np.random.shuffle(unique_cell_lines)
    folds = np.array_split(unique_cell_lines, 10)
    test_lines = folds[0]
    train_idxs = list(range(10))
    train_idxs.remove(n_fold)
    np.random.seed(420)
    validation_idx = np.random.choice(train_idxs)
    train_idxs.remove(validation_idx)
    train_lines = np.concatenate([folds[idx] for idx in train_idxs])
    validation_lines = folds[validation_idx]
    test_lines = folds[n_fold]
    train_data = data.query("SANGER_MODEL_ID in @train_lines")
    validation_data = data.query("SANGER_MODEL_ID in @validation_lines")
    test_data = data.query("SANGER_MODEL_ID in @test_lines")
    return (scripts.OmicsDataset(cell_dict, drug_dict, train_data),
    scripts.OmicsDataset(cell_dict, drug_dict, validation_data),
    scripts.OmicsDataset(cell_dict, drug_dict, test_data))

# Configuration

we declare the configuration, this is going to be model-specific and we get the datasets

In [4]:
config = {"features" : {"fp_radius":2},
          "optimizer": {"batch_size": 3,
                        "clip_norm":19,
                        "learning_rate":0.0004592646200179472,
                        "stopping_patience":15},
          "model":{"embed_dim":485,
                 "hidden_dim":696,
                 "dropout":0.48541242824674574,
                 "n_layers": 4,
                 "norm": "layernorm"},
         "env": {"fold": 0,
                 "device":"cpu",
                 "max_epochs": 100,
                 "search_hyperparameters":True}}

In [7]:
train_dataset, validation_dataset, test_dataset = get_data(n_fold = config["env"]["fold"],
                                                           fp_radius = config["features"]["fp_radius"])



[14:29:37] DEPRECATION WARNING: please use MorganGenerator
[14:29:37] DEPRECATION WARNING: please use MorganGenerator
[14:29:37] DEPRECATION WARNING: please use MorganGenerator
[14:29:37] DEPRECATION WARNING: please use MorganGenerator
[14:29:37] DEPRECATION WARNING: please use MorganGenerator
[14:29:37] DEPRECATION WARNING: please use MorganGenerator
[14:29:37] DEPRECATION WARNING: please use MorganGenerator
[14:29:37] DEPRECATION WARNING: please use MorganGenerator
[14:29:37] DEPRECATION WARNING: please use MorganGenerator
[14:29:37] DEPRECATION WARNING: please use MorganGenerator
[14:29:37] DEPRECATION WARNING: please use MorganGenerator
[14:29:37] DEPRECATION WARNING: please use MorganGenerator
[14:29:37] DEPRECATION WARNING: please use MorganGenerator
[14:29:37] DEPRECATION WARNING: please use MorganGenerator
[14:29:37] DEPRECATION WARNING: please use MorganGenerator
[14:29:37] DEPRECATION WARNING: please use MorganGenerator
[14:29:37] DEPRECATION WARNING: please use MorganGenerat

In [8]:
cell_features, drug_features, target, drug_index, cell_indices, labels = train_dataset[4]

ins = train_dataset[7]
print(ins[5].shape)
print(ins[4].shape)

print("Drug Features Shape:", drug_features.shape)
print("Cell Features Shape:", cell_features.shape)  
print("Labels Shape:", labels.shape)
print("Target Shape:", target.shape)               
print("Drug Index Shape:", drug_index.shape)      
print("Cell Indices Shape:", cell_indices.shape) 




torch.Size([325])
torch.Size([325])
Drug Features Shape: torch.Size([323, 2048])
Cell Features Shape: torch.Size([323, 777])
Labels Shape: torch.Size([323])
Target Shape: torch.Size([323])
Drug Index Shape: torch.Size([323])
Cell Indices Shape: torch.Size([323])


# Hyperparameter optimization

we wrap the function for training the model in a function that can be used by optuna

In [7]:
def train_model_optuna(trial, config):
    def pruning_callback(epoch, train_r):
        trial.report(train_r, step = epoch)
        if np.isnan(train_r):
            raise optuna.TrialPruned()
        if trial.should_prune():
            raise optuna.TrialPruned()
    config["model"] = {"embed_dim": trial.suggest_int("embed_dim", 64, 512),
                    "hidden_dim": trial.suggest_int("hidden_dim", 64, 2048),
                    "n_layers": trial.suggest_int("n_layers", 1, 6),
                    "norm": trial.suggest_categorical("norm", ["batchnorm", "layernorm", None]),
                    "dropout": trial.suggest_float("dropout", 0.0, 0.5)}
    config["optimizer"] = { "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-1, log=True),
                            "clip_norm": trial.suggest_int("clip_norm", 0.1, 20),
                            "batch_size": trial.suggest_int("batch_size", 5, 10),
                            "stopping_patience":10}
    try:
        R, model = scripts.train_model(config,
                                       train_dataset,
                                       validation_dataset,
                                       use_momentum=True,
                                       callback_epoch = pruning_callback)
        return R
    except Exception as e:
        print(e)
        return 0

In [8]:


if config["env"]["search_hyperparameters"]:
    study_name = f"baseline_model"
    storage_name = "sqlite:///studies/{}.db".format(study_name)
    study = optuna.create_study(study_name=study_name,
                                storage=storage_name,
                                direction='maximize',
                                load_if_exists=True,
                                pruner=optuna.pruners.MedianPruner(n_startup_trials=30,
                                                               n_warmup_steps=5,
                                                               interval_steps=5))
    objective = lambda x: train_model_optuna(x, config)
    study.optimize(objective, n_trials=40)
    best_config = study.best_params
    print(best_config)
    config["model"]["embed_dim"] = best_config["embed_dim"]
    config["model"]["hidden_dim"] = best_config["hidden_dim"]
    config["model"]["n_layers"] = best_config["n_layers"]
    config["model"]["norm"] = best_config["norm"]
    config["model"]["dropout"] = best_config["dropout"]
    config["optimizer"]["learning_rate"] = best_config["learning_rate"]
    config["optimizer"]["clip_norm"] = best_config["clip_norm"]
    config["optimizer"]["batch_size"] = best_config["batch_size"]



[I 2025-01-03 22:54:43,685] Using an existing study with name 'baseline_model' instead of creating a new one.
/home/karolyi/miniconda/envs/project_thesis/lib/python3.13/site-packages/torchmetrics/utilities/prints.py:43: FutureWarning: The default value for `maximize` will be changed from `True` to `None` in v1.7.0 of TorchMetrics,will automatically infer the value based on the `higher_is_better` attribute of the metric (if such attribute exists) or raise an error if it does not. If you are explicitly setting the `maximize` argument to either `True` or `False` already, you can ignore this warning.
  warnings.warn(*args, **kwargs)  # noqa: B028


stack expects each tensor to be equal size, but got [700, 777] at entry 0 and [746, 777] at entry 1


[I 2025-01-03 22:54:47,174] Trial 324 finished with value: 0.0 and parameters: {'embed_dim': 395, 'hidden_dim': 1749, 'n_layers': 5, 'norm': 'layernorm', 'dropout': 0.4697537976927172, 'learning_rate': 2.2133210140554165e-06, 'clip_norm': 19, 'batch_size': 10}. Best is trial 0 with value: 0.0.
[I 2025-01-03 22:54:47,888] Trial 325 finished with value: 0.0 and parameters: {'embed_dim': 334, 'hidden_dim': 1963, 'n_layers': 3, 'norm': None, 'dropout': 0.04134664710613227, 'learning_rate': 4.9817716094460155e-06, 'clip_norm': 1, 'batch_size': 10}. Best is trial 0 with value: 0.0.


stack expects each tensor to be equal size, but got [705, 777] at entry 0 and [728, 777] at entry 1


[I 2025-01-03 22:54:48,490] Trial 326 finished with value: 0.0 and parameters: {'embed_dim': 351, 'hidden_dim': 1785, 'n_layers': 1, 'norm': 'layernorm', 'dropout': 0.07245273030883095, 'learning_rate': 4.107238231780346e-05, 'clip_norm': 10, 'batch_size': 10}. Best is trial 0 with value: 0.0.


...

stack expects each tensor to be equal size, but got [618, 777] at entry 0 and [323, 777] at entry 1


[I 2025-01-03 22:55:13,344] Trial 362 finished with value: 0.0 and parameters: {'embed_dim': 401, 'hidden_dim': 1587, 'n_layers': 4, 'norm': 'layernorm', 'dropout': 0.34136277769870843, 'learning_rate': 1.0874372501186351e-05, 'clip_norm': 2, 'batch_size': 10}. Best is trial 0 with value: 0.0.


stack expects each tensor to be equal size, but got [688, 777] at entry 0 and [736, 777] at entry 1
stack expects each tensor to be equal size, but got [737, 777] at entry 0 and [328, 777] at entry 1


[I 2025-01-03 22:55:14,095] Trial 363 finished with value: 0.0 and parameters: {'embed_dim': 303, 'hidden_dim': 713, 'n_layers': 4, 'norm': None, 'dropout': 0.11558132678015265, 'learning_rate': 3.641299640383785e-06, 'clip_norm': 9, 'batch_size': 9}. Best is trial 0 with value: 0.0.


{'embed_dim': 331, 'hidden_dim': 1830, 'n_layers': 4, 'norm': 'layernorm', 'dropout': 0.045331257241234324, 'learning_rate': 1.4053783862508277e-06, 'clip_norm': 8, 'batch_size': 305}


# Model training and evaluation

After we have a set of optimal hyperparameters we train our model. The train model function could be changed, but:
- test_dataset cannot be used until we call the final evaluation step
- the evaluation step cannot be modified, it must take the model produced by your pipeline, a dataloader that provides the correct data for your model, and the final metrics have to be printed

In [9]:
def custom_collate_fn(batch):
    # Determine the maximum length in the batch
    max_len = max(item[4].size(0) for item in batch)  
    #print(max_len)

    padded_batch = []
    for cell_features, drug_features, target, drug_index, cell_indices, labels in batch:
        pad_len = max_len - cell_features.size(0)

        # Pad Cell Features to match max_len (rows only)
        padded_cell_features = torch.nn.functional.pad(cell_features, (0, 0, 0, pad_len))  # Pad rows
        padded_drug_feature = torch.nn.functional.pad(drug_features, (0, 0, 0, pad_len))

        # Pad associated vectors to match max_len
        padded_labels = torch.nn.functional.pad(labels, (0, pad_len))
        padded_target = torch.nn.functional.pad(target, (0, pad_len))
        padded_drug_index = torch.nn.functional.pad(drug_index, (0, pad_len))
        padded_cell_indices = torch.nn.functional.pad(cell_indices, (0, pad_len))

        padded_batch.append((padded_cell_features, padded_drug_feature, padded_target,
                             padded_drug_index, padded_cell_indices, padded_labels))

    # Stack all tensors
    cell_features = torch.stack([item[0] for item in padded_batch])
    drug_features = torch.stack([item[1] for item in padded_batch])
    target = torch.stack([item[2] for item in padded_batch])
    drug_index = torch.stack([item[3] for item in padded_batch])
    cell_indices = torch.stack([item[4] for item in padded_batch])
    labels = torch.stack([item[5] for item in padded_batch])

    return cell_features, drug_features, target, drug_index, cell_indices, labels


In [10]:
import scripts

%reload_ext autoreload

%autoreload 2

_, model = scripts.train_model(config, train_dataset,  validation_dataset=None, use_momentum=True, callback_epoch=None, collate_fn=custom_collate_fn)
device = torch.device(config["env"]["device"])
metrics = torchmetrics.MetricTracker(torchmetrics.MetricCollection(
    {"R_cellwise_residuals":scripts.GroupwiseMetric(metric=torchmetrics.functional.pearson_corrcoef,
                          grouping="drugs",
                          average="macro",
                          residualize=True),
    "R_cellwise":scripts.GroupwiseMetric(metric=torchmetrics.functional.pearson_corrcoef,
                          grouping="cell_lines",
                          average="macro",
                          residualize=False),
    "MSE":torchmetrics.MeanSquaredError()}))
metrics.to(device)
test_dataloader = torch.utils.data.DataLoader(test_dataset,
                                       batch_size=config["optimizer"]["batch_size"],
                                       drop_last=False,
                                      shuffle=False,
                                      collate_fn=custom_collate_fn)


/home/karolyi/miniconda/envs/project_thesis/lib/python3.13/site-packages/torchmetrics/utilities/prints.py:43: FutureWarning: The default value for `maximize` will be changed from `True` to `None` in v1.7.0 of TorchMetrics,will automatically infer the value based on the `higher_is_better` attribute of the metric (if such attribute exists) or raise an error if it does not. If you are explicitly setting the `maximize` argument to either `True` or `False` already, you can ignore this warning.
  warnings.warn(*args, **kwargs)  # noqa: B028


epoch : 0: train loss: 7.117366836621211 Smoothed R interaction (validation) None
epoch : 1: train loss: 6.5399534822656555 Smoothed R interaction (validation) None
epoch : 2: train loss: 5.41440790777023 Smoothed R interaction (validation) None
epoch : 3: train loss: 4.078846876437847 Smoothed R interaction (validation) None
epoch : 4: train loss: 2.742113577058682 Smoothed R interaction (validation) None
epoch : 5: train loss: 2.1864217943870106 Smoothed R interaction (validation) None
epoch : 6: train loss: 2.1299695263688383 Smoothed R interaction (validation) None
epoch : 7: train loss: 1.7901446292033563 Smoothed R interaction (validation) None
epoch : 8: train loss: 1.7497069870050137 Smoothed R interaction (validation) None
epoch : 9: train loss: 1.6424014528210347 Smoothed R interaction (validation) None
epoch : 10: train loss: 1.6347753818218524 Smoothed R interaction (validation) None
epoch : 11: train loss: 1.6087781872886877 Smoothed R interaction (validation) None
epoch :

In [11]:
final_metrics = scripts.evaluate_step(model, test_dataloader, metrics, device)
print(final_metrics)

RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 41 but got size 42 for tensor number 1 in the list.